In [1]:
import nibabel as nib
import numpy as np
import os
from scipy.signal import find_peaks

# Squeezing out the empty slices

Remove all slices that all pixels in that slice have value below 'background_value'

In [2]:
def filter_slices(image, axis, background_value = -4000):
    non_background_slices = []
    for i in range(image.shape[axis]):
        if axis == 0:
            slice_2d = image[i, :, :]
        elif axis == 1:
            slice_2d = image[:, i, :]
        elif axis == 2:
            slice_2d = image[:, :, i]
        
        if not np.all(slice_2d <= background_value):
            non_background_slices.append(i)
    
    if axis == 0:
        filtered_image = image[non_background_slices, :, :]
    elif axis == 1:
        filtered_image = image[:, non_background_slices, :]
    elif axis == 2:
        filtered_image = image[:, :, non_background_slices]
    
    return filtered_image

In [3]:
# expected: input_vol = nib.load([path_to_.nii.gz file])

def filter_vol(input_vol, background_value):
    vol = input_vol.get_fdata()
    
    filtered_image = filter_slices(vol, 0, background_value)
    filtered_image = filter_slices(filtered_image, 1, background_value)
    filtered_image = filter_slices(filtered_image, 2, background_value)

    output_vol = nib.Nifti1Image(filtered_image, input_vol.affine, input_vol.header)

    return output_vol

# Flip R to L

In [4]:
# expected: input_vol = nib.load([path_to_.nii.gz file])

def flip_vol_R_to_L(input_vol):
    vol = input_vol.get_fdata()

    flipped_vol = np.zeros_like(vol)
    for slice_number in range(vol.shape[2]):
        flipped_vol[:,:,slice_number] = np.flip(vol[:,:,slice_number],0)
    
    output_vol = nib.Nifti1Image(flipped_vol, input_vol.affine, input_vol.header)

    return output_vol

In [5]:
# expected: input_path = r"C:\Users\acer\Desktop\TMJOA\Masked\47-16872 L_masked.nii.gz"

def check_for_L_or_R(input_path) :
    file_name = os.path.basename(input_path)

    # Check if "R_" or "L_" is in the file name
    if "R" in file_name:
        return 'R'
    elif "L" in file_name:
        return 'L'
    else:
        raise ValueError("File name does not contain 'R_' or 'L_'")

# Crop the condyle

Crop the condyle at the lowest point in the bone curve

In [6]:
def local_minima(data):
    # Inverting the data to find local minima as local maxima
    inverted_data = -np.array(data)
    
    # Finding local maxima in the inverted data
    peaks, _ = find_peaks(inverted_data)
    
    if len(peaks) > 1:
        filtered_peak = [value for value in peaks if value > 100]
        peaks = [min(filtered_peak)]
    if len(peaks) == 0:
        peaks = [150]

    peak_pos = peaks[0]
    #peak_value = data[peak_pos]
    return peak_pos#, peak_value

In [7]:
def first_non_background_column(arr, background=-3990):

    for col_index in range(arr.shape[1]):
        if np.any(arr[:, col_index] > background):
            return col_index
        
    return -1

In [8]:
# expected: input_vol = nib.load(input_path)

def cropping_TMJ(input_vol):
    
    vol = input_vol.get_fdata()

    top = []

    for slice_number in range(vol.shape[1]):
        slice_data = vol[:, slice_number, :]
        top_pixel = first_non_background_column(slice_data, background=-3900)
        top.append(top_pixel)

    flip_top = max(top) - np.array(top)
    peak_pos = local_minima(flip_top)

    cutting_axis_1 = peak_pos
    cutting_axis_2 = first_non_background_column(vol[:,peak_pos,:], background=-3900) + 50

    result_vol = vol[:,0:cutting_axis_1, 0:cutting_axis_2]
    result_img = nib.Nifti1Image(result_vol, input_vol.affine, input_vol.header)

    return result_img

# Cropp down to 224, 224, 224

Crop down to dimensions 224, 224, 224 while making sure to preserve the condylar head

In [9]:
# expected: input_vol = nib.load(input_path)

def resize_nifti(input_vol, target_shape=(224, 224, 224), padding_value=-4000, background_threshold=-3000):
    # Load the NIfTI file
    data = input_vol.get_fdata()

    # Get the current shape
    current_shape = data.shape
    
    # Initialize the output array with the padding value
    output_data = np.full(target_shape, padding_value)
    
    # Dimension 0: Cropping and padding while preserving non-background voxels in [:, 0:100, :]
    if current_shape[0] > target_shape[0]:
        # Find the range of non-background voxels in [:, 0:100, :]
        non_bg_indices = np.where(data[:, 0:100, :] > background_threshold)[0]
        if len(non_bg_indices) > 0:
            start_preserve = non_bg_indices.min()
            end_preserve = non_bg_indices.max() + 1
            
            # Calculate potential crop
            total_crop = current_shape[0] - target_shape[0]
            start_crop = max(0, start_preserve - (total_crop // 2))
            end_crop = min(current_shape[0], start_crop + target_shape[0])
            
            # Adjust if end_crop removes non-background voxels
            if end_crop < end_preserve:
                end_crop = min(current_shape[0], end_preserve)
                start_crop = max(0, end_crop - target_shape[0])
            
            # Perform cropping
            dim0_data = data[start_crop:end_crop, :, :]
            
            # Pad if necessary (in case we couldn't crop fully)
            if dim0_data.shape[0] < target_shape[0]:
                pad_before = (target_shape[0] - dim0_data.shape[0]) // 2
                pad_after = target_shape[0] - dim0_data.shape[0] - pad_before
                dim0_data = np.pad(dim0_data, ((pad_before, pad_after), (0, 0), (0, 0)), 
                                   mode='constant', constant_values=padding_value)
        else:
            # If no non-background voxels, crop from center
            start = (current_shape[0] - target_shape[0]) // 2
            end = start + target_shape[0]
            dim0_data = data[start:end, :, :]
    else:
        # Pad if smaller
        pad_before = (target_shape[0] - current_shape[0]) // 2
        pad_after = target_shape[0] - current_shape[0] - pad_before
        dim0_data = np.pad(data, ((pad_before, pad_after), (0, 0), (0, 0)), 
                           mode='constant', constant_values=padding_value)

    # Dimension 1: Cropping and padding at the end
    if dim0_data.shape[1] > target_shape[1]:
        dim1_data = dim0_data[:, :target_shape[1], :]
    else:
        pad_width = target_shape[1] - dim0_data.shape[1]
        dim1_data = np.pad(dim0_data, ((0, 0), (0, pad_width), (0, 0)), 
                           mode='constant', constant_values=padding_value)

    # Dimension 2: Same as dimension 1
    if dim1_data.shape[2] > target_shape[2]:
        output_data = dim1_data[:, :, :target_shape[2]]
    else:
        pad_width = target_shape[2] - dim1_data.shape[2]
        output_data = np.pad(dim1_data, ((0, 0), (0, 0), (0, pad_width)), 
                             mode='constant', constant_values=padding_value)

    # Create a new NIfTI image with the resized data
    output_vol = nib.Nifti1Image(output_data, input_vol.affine, input_vol.header)
    
    # Save the resized image
    return output_vol

# Main

In [10]:
input_folder = r"D:\Kananat\Masked_dim1_expand0px"
output_folder = r"D:\Kananat\Preprocessed_expand0px"

nii_count = len([filename for filename in os.listdir(input_folder) if filename.endswith('.nii.gz')])
print(f"There are {nii_count} .nii files in the {input_folder}")

progress_count = 0

files = sorted(os.listdir(input_folder))

for filename in files :
    if filename.endswith('.nii.gz'):
        progress_count += 1
        print(f"[Processing {progress_count} out of {nii_count}]")

        input_path = os.path.join(input_folder, filename)

        output_filename = filename.split("_")[0]
        output_filename = f"{output_filename}.nii.gz"
        output_path = os.path.join(output_folder, output_filename)

        if os.path.exists(input_path) :

            input_vol = nib.load(input_path)

            output_vol = filter_vol(input_vol, background_value = -4000)
            if check_for_L_or_R(input_path) == 'R':
                output_vol = flip_vol_R_to_L(output_vol)
            output_vol = cropping_TMJ(output_vol)
            output_vol = resize_nifti(output_vol, target_shape=(224, 224, 224), padding_value=-4000, background_threshold=-3000)

            nib.save(output_vol, output_path)
        else:
            vol_exist = os.path.exists(input_path)
            print(f"Error : vol {vol_exist}")

print("Done")

There are 365 .nii files in the D:\Kananat\Masked_dim1_expand0px
[Processing 1 out of 365]
[Processing 2 out of 365]
[Processing 3 out of 365]
[Processing 4 out of 365]
[Processing 5 out of 365]
[Processing 6 out of 365]
[Processing 7 out of 365]
[Processing 8 out of 365]
[Processing 9 out of 365]
[Processing 10 out of 365]
[Processing 11 out of 365]
[Processing 12 out of 365]
[Processing 13 out of 365]
[Processing 14 out of 365]
[Processing 15 out of 365]
[Processing 16 out of 365]
[Processing 17 out of 365]
[Processing 18 out of 365]
[Processing 19 out of 365]
[Processing 20 out of 365]
[Processing 21 out of 365]
[Processing 22 out of 365]
[Processing 23 out of 365]
[Processing 24 out of 365]
[Processing 25 out of 365]
[Processing 26 out of 365]
[Processing 27 out of 365]
[Processing 28 out of 365]
[Processing 29 out of 365]
[Processing 30 out of 365]
[Processing 31 out of 365]
[Processing 32 out of 365]
[Processing 33 out of 365]
[Processing 34 out of 365]
[Processing 35 out of 365]